In [1]:
from matplotlib import pyplot as plt
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

c:\users\shivangi pandey\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pickle_file = 'complete_dataset.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train1_dataset = save['train_dataset']
    train1_labels = save['train_labels']
    valid1_dataset = save['valid_dataset']
    valid1_labels = save['valid_labels']
    test1_dataset = save['test_dataset']
    test1_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train1_dataset.shape, train1_labels.shape)
    print('Validation set', valid1_dataset.shape, valid1_labels.shape)
    print('Test set', test1_dataset.shape, test1_labels.shape)
    


Training set (45931, 32, 32) (45931,)
Validation set (15268, 32, 32) (15268,)
Test set (15268, 32, 32) (15268,)


In [3]:
image_size = 32
#image_size = 64
num_labels = 62
num_channels = 1

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels


train_dataset, train_labels = reformat(train1_dataset, train1_labels)
valid_dataset, valid_labels = reformat(valid1_dataset, valid1_labels)
test_dataset, test_labels = reformat(test1_dataset, test1_labels)
print('Training set', train_dataset.shape,train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)



Training set (45931, 32, 32, 1) (45931, 62)
Validation set (15268, 32, 32, 1) (15268, 62)
Test set (15268, 32, 32, 1) (15268, 62)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [10]:
batch_size = 1000
# 500 acc - 82.1 with overfitting
patch_size = 5
depth = 16
num_hidden = 64
beta = 0.01



graph = tf.Graph()

with graph.as_default():

    # Input data 
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(None, image_size, image_size, num_channels),name = 'tf_train_dataset')
    tf_train_labels = tf.placeholder( 
        tf.float32, shape=(None, num_labels),name='tf_train_labels')
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    layer1_weights = tf.Variable(tf.truncated_normal(
        [5,5,num_channels,20], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([20]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [5,5,20,50], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0,shape=[50]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [5*5*50, 500], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[500]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [500, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding='VALID')
        max_pool = tf.nn.max_pool(conv,ksize=[1,2,2,1],strides=[1,2,2,1], padding='VALID')
        hidden = tf.nn.relu(max_pool + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
        max_pool = tf.nn.max_pool(conv,ksize=[1,2,2,1],strides=[1,2,2,1], padding='VALID') 
        hidden = tf.nn.relu(max_pool + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden,[-1,shape[1]*shape[2]*shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape,layer3_weights) + layer3_biases)
        #keep_prob = 0.8
        #relu_layer_dropout = tf.nn.dropout(hidden,keep_prob)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # Training computation
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    reg_constant = 1.5  # Choose an appropriate one.
    #1.0 acc- 83.7 with overfitting
    loss = loss + reg_constant * sum(reg_losses)
    
    #Optimizer
    #optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
    optimizer = tf.train.MomentumOptimizer(0.001,momentum=0.9).minimize(loss)
    #optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.001,rho=0.95,epsilon=1e-08,use_locking=False,name='Adadelta').minimize(loss)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001,beta1=0.9,beta2=0.999,epsilon=1e-08,use_locking=False,name='Adam').minimize(loss)
    #optimizer = tf.train.AdagradOptimizer(0.01,initial_accumulator_value=0.1,use_locking=False,name='Adagrad').minimize(loss)
    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits,name = 'train_prediction')
    #print(train_prediction.shape)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
   
    

In [11]:
num_steps = 1500
# 1650 acc - 83.7 with overfitting
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    saver = tf.train.Saver()
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
        if(step % 500 == 0):
            saver.save(session, './my_text_recognizer_model')
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    values = np.argmax(test_prediction.eval(), 1) == np.argmax(test_labels, 1)
    true_Values = np.count_nonzero(values == True)
    print("true values : ",true_Values)
    print("False values : ",len(values) - true_Values)
    
    

Initialized
Minibatch loss at step 0: 12.414981
Minibatch accuracy: 1.3%
Validation accuracy: 1.4%
Minibatch loss at step 50: 3.165189
Minibatch accuracy: 27.6%
Validation accuracy: 28.7%
Minibatch loss at step 100: 1.609526
Minibatch accuracy: 59.6%
Validation accuracy: 60.7%
Minibatch loss at step 150: 1.303228
Minibatch accuracy: 66.4%
Validation accuracy: 66.3%
Minibatch loss at step 200: 1.095687
Minibatch accuracy: 70.6%
Validation accuracy: 69.4%
Minibatch loss at step 250: 1.088568
Minibatch accuracy: 71.5%
Validation accuracy: 72.0%
Minibatch loss at step 300: 0.908129
Minibatch accuracy: 75.0%
Validation accuracy: 73.6%
Minibatch loss at step 350: 0.877928
Minibatch accuracy: 76.4%
Validation accuracy: 74.6%
Minibatch loss at step 400: 0.713520
Minibatch accuracy: 79.8%
Validation accuracy: 75.4%
Minibatch loss at step 450: 0.684192
Minibatch accuracy: 78.8%
Validation accuracy: 76.9%
Minibatch loss at step 500: 0.693568
Minibatch accuracy: 80.1%
Validation accuracy: 78.3%
Mi